# Mathematical Lifts: От дискретного к непрерывному

В этом туториале мы изучим **математические подъёмы** — преобразование дискретных систем переписывания в непрерывные математические структуры.

## Компоненты L7

1. 📐 **Геометрия**: Погружение в ℝⁿ, многообразия, метрики
2. 📊 **Меры**: Вероятностные меры, энтропия, инвариантность
3. 🎯 **Категории**: Функторы, морфизмы, естественные преобразования

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from rewriting import String, Rule, RewritingEngine

print("✓ Модули загружены")

✓ Модули загружены


## 1. Геометрические подъёмы

Преобразуем строки в точки многомерного пространства.

In [2]:
from math_lifts.geometry import GeometricLift, MetricSpace, Point

# Погружение в ℝ⁸
lift = GeometricLift(embedding_dim=8)

# Несколько строк
strings = [
    String.from_str("00"),
    String.from_str("||"),
    String.from_str("0|0"),
    String.from_str("|"),
]

# Погружаем в ℝ⁸
points = [lift.embed_string(s) for s in strings]

print("Погружение строк в ℝ⁸:")
for s, p in zip(strings, points):
    print(f"  {s} → Point({p.coordinates[:3]}...) [размерность: {len(p.coordinates)}]")

Погружение строк в ℝ⁸:
  00 → Point((0.02, 1.0, 0.0)...) [размерность: 8]
  || → Point((0.02, 0.0, 1.0)...) [размерность: 8]
  0|0 → Point((0.03, 0.6666666666666666, 0.3333333333333333)...) [размерность: 8]
  | → Point((0.01, 0.0, 1.0)...) [размерность: 8]


### Метрическое пространство

Используем расстояние Левенштейна между строками:

In [3]:
metric = MetricSpace()

s1 = String.from_str("00")
s2 = String.from_str("0|")
s3 = String.from_str("||")

print("Матрица расстояний (Левенштейн):")
print(f"  d({s1}, {s2}) = {metric.distance(s1, s2)}")
print(f"  d({s1}, {s3}) = {metric.distance(s1, s3)}")
print(f"  d({s2}, {s3}) = {metric.distance(s2, s3)}")

Матрица расстояний (Левенштейн):
  d(00, 0|) = 1.0
  d(00, ||) = 2.0
  d(0|, ||) = 1.0


### Многообразия

Исследуем геометрию множества строк:

In [4]:
from math_lifts.geometry import Manifold

# Создаём многообразие из точек
manifold = Manifold(points[:4], metric)

# Геометрические характеристики
dimension = manifold.dimension  # Вычисляется автоматически в __post_init__
curvature_val = manifold.curvature()  # Вызываем метод

print("Характеристики многообразия:")
print(f"  Размерность: {dimension}")
print(f"  Кривизна: {curvature_val:.4f}")
print(f"  Точек: {len(points)}")

Характеристики многообразия:
  Размерность: 1
  Кривизна: 0.2477
  Точек: 4


### Геодезические

Кратчайший путь между двумя строками:

In [5]:
# Геодезическая между двумя строками
start = String.from_str("0000")
end = String.from_str("||")

geodesic = lift.compute_geodesic(start, end, steps=5)

print(f"Геодезическая от '{start}' до '{end}':")
print(f"  Интерполяционные точки: {len(geodesic)}")
print(f"  Первая точка: {geodesic[0].coordinates[:3]}...")
print(f"  Последняя точка: {geodesic[-1].coordinates[:3]}...")

Геодезическая от '0000' до '||':
  Интерполяционные точки: 6
  Первая точка: (0.04, 1.0, 0.0)...
  Последняя точка: (0.02, 0.0, 1.0)...


## 2. Теоретико-мерные подъёмы

Изучаем вероятностные меры и энтропию.

In [6]:
from math_lifts.measure import Measure, Probability, FrequencyMeasure

# Простая мера
measure = Measure({
    String.from_str("0"): 2.0,
    String.from_str("|"): 3.0,
    String.from_str("00"): 1.5,
})

print("Мера на строках:")
print(f"  μ('0') = {measure(String.from_str('0'))}")
print(f"  μ('|') = {measure(String.from_str('|'))}")
print(f"  Полная мера: {measure.total()}")
print()

# Нормализация в вероятностную меру
prob = measure.normalize()
print("Вероятностная мера:")
print(f"  P('0') = {prob(String.from_str('0')):.3f}")
print(f"  P('|') = {prob(String.from_str('|')):.3f}")
print(f"  Энтропия: {prob.entropy():.3f} бит")

Мера на строках:
  μ('0') = 2.0
  μ('|') = 3.0
  Полная мера: 6.5

Вероятностная мера:
  P('0') = 0.308
  P('|') = 0.462
  Энтропия: 1.526 бит


### Частотная мера

Строим эмпирическую меру из наблюдений:

In [7]:
# Создаём систему переписывания
rules = [
    Rule(String.from_str("00"), String.from_str("|")),
    Rule(String.from_str("||"), String.from_str("0")),
]
engine = RewritingEngine(rules)

# Собираем статистику
initial = String.from_str("0000")
reachable_levels = engine.bounded_reach(initial, depth=8)

# Собираем все строки из всех уровней
reachable = set()
for level_strings in reachable_levels.values():
    reachable.update(level_strings)

freq = FrequencyMeasure()
freq.observe_many(list(reachable))

empirical = freq.get_probability()

print(f"Эмпирическая мера ({freq.total_count} наблюдений):")
print(f"  Энтропия: {empirical.entropy():.3f} бит")
print(f"  Размер носителя: {len(empirical.support())}")

Эмпирическая мера (6 наблюдений):
  Энтропия: 2.585 бит
  Размер носителя: 6


### Инвариантная мера

Мера, сохраняющаяся под действием системы:

In [8]:
from math_lifts.measure import InvariantMeasure

inv = InvariantMeasure(rules)

# Поиск инвариантной меры
strings_list = list(reachable)[:20]
inv_measure = inv.find_invariant(strings_list, iterations=30)

if inv_measure:
    print("Инвариантная мера найдена:")
    print(f"  Энтропия: {inv_measure.entropy():.3f} бит")
    
    # Топ-5 строк по вероятности
    top_strings = sorted(
        [(s, inv_measure(s)) for s in strings_list],
        key=lambda x: x[1],
        reverse=True
    )[:5]
    
    print("  Топ-5 по вероятности:")
    for s, p in top_strings:
        if p > 0.01:
            print(f"    {s}: {p:.3f}")

Инвариантная мера найдена:
  Энтропия: 0.764 бит
  Топ-5 по вероятности:
    0: 0.778
    0|0: 0.222


## 3. Категорные структуры

Абстрагируем систему в категорию с функторами.

In [9]:
from math_lifts.category import RewritingCategory, Object, Morphism

# Создаём категорию переписывания
cat = RewritingCategory(rules)

# Добавляем объекты (строки)
s1 = String.from_str("00")
s2 = String.from_str("|")
s3 = String.from_str("0")

obj1 = cat.get_object(s1)
obj2 = cat.get_object(s2)
obj3 = cat.get_object(s3)

print("Объекты категории:")
print(f"  {obj1}")
print(f"  {obj2}")
print(f"  {obj3}")
print()

# Добавляем морфизмы (применения правил)
cat.add_rewriting_step(s1, s2, rules[0])

print(f"Категория: {cat.category}")

Объекты категории:
  00 ∈ Rewriting
  | ∈ Rewriting
  0 ∈ Rewriting

Категория: Category Rewriting: |Ob| = 3, |Mor| = 1


### Функторы

Функтор геометрического подъёма переводит строки в точки:

In [10]:
from math_lifts.category import Category, GeometricFunctor

# Категория геометрии
geometry_cat = Category("Geometry")

# Функтор F: Rewriting → Geometry
functor = GeometricFunctor(
    cat.category,
    geometry_cat,
    lift
)

print(f"Функтор: {functor}")
print()

# Действие на объект
F_obj1 = functor.map_object(obj1)
print(f"F({obj1.name}) = {F_obj1.name}")
print(f"  Данные: Point в ℝ{lift.embedding_dim}")

Функтор: GeomLift: Rewriting → Geometry

F(00) = F(00)
  Данные: Point в ℝ8


## 📊 Резюме

В этом туториале вы изучили:

✅ **Геометрию**: Погружение в ℝⁿ, метрики, многообразия, геодезические  
✅ **Меры**: Вероятностные меры, энтропия, инвариантность  
✅ **Категории**: Объекты, морфизмы, функторы  

## 🎯 Ключевые концепции

- Дискретные структуры можно погрузить в непрерывные пространства
- Метрические пространства позволяют измерять "близость" строк
- Вероятностные меры описывают распределения
- Категории абстрагируют структуру преобразований
- Функторы связывают разные уровни абстракции

## 🚀 Следующий шаг

**03_full_integration.ipynb** — Полный пайплайн через `SpaceLanguageOrchestrator`